<a href="https://colab.research.google.com/github/eckoecho/CodingDojo/blob/main/Project_1_Part_5_(Core).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
#Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn import set_config
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
#Ensure output is in pandas form
set_config(transform_output="pandas")

In [161]:
#Load data
fpath="/content/drive/MyDrive/CodingDojo/02-MachineLearning/Week05/Data/sales_predictions_2023.csv"
df = pd.read_csv(fpath)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


## Drop Duplicate

In [162]:
#Count of duplicates in our df
df.duplicated().sum()

0

## Fix inconsistencies in categorical data

In [163]:
#Getting categorical columns
cat_cols = df.select_dtypes("object").columns
cat_cols

Index(['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')

### Categorical Columns Cleaning

In [164]:
#Fixing string inconsistencies
#First, obtaining all features with string values
str_cols = df.select_dtypes(include="object").columns
str_cols

Index(['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')

In [165]:
#Looping thru each column and getting value_counts to see out of place values in each feature
for col in str_cols:
    print(f"Value counts for {col}: ")
    print(df[col].value_counts())  
    #Increase readibility with new lines between results
    print("\n")


Value counts for Item_Identifier: 
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64


Value counts for Item_Fat_Content: 
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64


Value counts for Item_Type: 
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64


Value counts for Outlet_Identifier: 
OUT027    935
OUT013    932
OUT049    930
OUT046

#### Cleaning `Item_Fat_Content`

In [166]:
rename_dict = {"low fat": "Low Fat",
               "LF": "Low Fat",
               "reg": "Regular"}

In [167]:
#Item_Fat_Content values are inconsistent, going to standardize with .str.replace
#Convert using .str.replace("old","new")
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace(rename_dict, regex=True)
df['Item_Fat_Content'].value_counts()
#We now have standardized our Item_Fat_Content feature

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

#### Cleaning `Outlet_Size`

In [168]:
#Converting outlet size of "high" to "large"
df["Outlet_Size"].replace({"High":"Large"}, inplace=True)
df["Outlet_Size"].value_counts()

Medium    2793
Small     2388
Large      932
Name: Outlet_Size, dtype: int64

## Identify the features (X) and target (y)

In [169]:
#Stating features
X = df.drop(columns="Item_Outlet_Sales")
#Setting target
y = df["Item_Outlet_Sales"]

## Perform a train test split

In [170]:
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Create a preprocessing object

In [171]:
# Initialize the preprocessing object
scaler = StandardScaler()
encoder = OneHotEncoder()

## Imputation of missing values

In [172]:
# Get the list of numeric column names 
num_cols = X_train.select_dtypes("number").columns
num_cols

Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')

In [173]:
# Null values in numeric columns of training data
null_count = X_train[num_cols].isna().sum()
null_count

Item_Weight                  1107
Item_Visibility                 0
Item_MRP                        0
Outlet_Establishment_Year       0
dtype: int64

In [174]:
#Seeing how many nulls in X_test
X_test.isna().sum()

Item_Identifier                0
Item_Weight                  356
Item_Fat_Content               0
Item_Visibility                0
Item_Type                      0
Item_MRP                       0
Outlet_Identifier              0
Outlet_Establishment_Year      0
Outlet_Size                  598
Outlet_Location_Type           0
Outlet_Type                    0
dtype: int64

In [175]:
# Instantiate the imputer object from the SimpleImputer class with strategy 'median'
impute_median = SimpleImputer(strategy='median')

In [176]:
# Fit the imputer object on the numeric training data with .fit() 
impute_median.fit(X_train[num_cols])

SimpleImputer(strategy='median')

In [177]:
# Transform the training data (fill in nulls with median)
X_train_num_imputed = impute_median.transform(X_train[num_cols])
X_train_num_imputed

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
4776,16.350,0.029565,256.4646,2009.0
7510,15.250,0.000000,179.7660,2009.0
5828,12.350,0.158716,157.2946,1999.0
5327,7.975,0.014628,82.3250,2004.0
4810,19.350,0.016645,120.9098,2002.0
...,...,...,...,...
5734,9.395,0.286345,139.1838,1998.0
5191,15.600,0.117575,75.6670,2007.0
5390,17.600,0.018944,237.3590,2002.0
860,20.350,0.054363,117.9466,2007.0


In [178]:
# Check for null values in numeric columns of trainingset
X_train_num_imputed.isna().sum()

Item_Weight                  0
Item_Visibility              0
Item_MRP                     0
Outlet_Establishment_Year    0
dtype: int64

In [179]:
# Get the list of categorical and ordinal column names 
cat_cols = X_train.select_dtypes("object").columns
cat_cols

Index(['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')

In [180]:
# Instantiate the new imputer
impute_na = SimpleImputer(strategy='constant', fill_value='NA')

In [181]:
# Fit the imputer on the training data
impute_na.fit(X_train[cat_cols])

SimpleImputer(fill_value='NA', strategy='constant')

In [182]:
# Check for missing values in cat columns
X_train[cat_cols].isna().sum().sum()

1812

In [183]:
# Transform the categorical training data
X_train_cat_imputed = impute_na.transform(X_train[cat_cols])
X_test_cat_imputed = impute_na.transform(X_test[cat_cols])

In [184]:
# Confirm that all null vales in cat columns are filled in training set
X_train_cat_imputed.isna().sum().sum()

0

In [185]:
# Confirm that all null vales in cat columns are filled in testing set
X_test_cat_imputed.isna().sum().sum()

0